In [16]:
from scipy.optimize import minimize
from scipy.linalg import toeplitz
import numpy as np

# general imports
import numpy as np
import pandas as pd
from math import ceil
from scipy import linalg
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_spd_matrix
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from matplotlib import pyplot

#for SKlearn compliance
from sklearn.base import BaseEstimator, RegressorMixin

In [17]:
n = 200
p= 1200

In [18]:
beta_star = np.concatenate(([1]*7, [0]*25, [0.25]*5, [0]*50, [0.7]*15, [0]*1098))

In [19]:
#The goal is to detect the position of the actual information ("signal"), where the beta are none-zero
print(beta_star)
print(np.where(beta_star != 0)) #indices of non-zero betas

[1. 1. 1. ... 0. 0. 0.]
(array([  0,   1,   2,   3,   4,   5,   6,  32,  33,  34,  35,  36,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101], dtype=int64),)


Form of the generated data:

$$ y = x\beta^* + \sigma \epsilon $$



In [20]:
#Toeplitz correlation matrix of form p^(|i-j|) and p=0.8
#Want toeplitz([1,0.8,0.8**2,0.8**3,...0.8**1199])
#toeplitz([1,2,3,4,5])
M = []
for i in range(p):
    M.append(0.8**i)

In [21]:
# The desired mean values of the sample.
n = 200
p= 1200

mu = [0]*p
sigma = 3.5
np.random.seed(123)

v = []
for i in range(p):
    v.append(0.8**i) #1D array (vector) of 0.8**i


r = toeplitz(v)

# Generate the random samples.
X = np.random.multivariate_normal(mu, r, size=n) #make some fake data
y = X.dot(beta_star) + sigma*np.random.normal(loc=0,scale=1,size=n) #actual target plus st.d times a random.normal noise
#Note that np.matmul (matrix multiplication) gives you a numpy array

In [22]:
# Testing the np.random.normal function
# print(np.random.normal(loc=0,scale=1,size=n).reshape(-1,1))

In [23]:
print(X.shape) #Woot!
print(y.shape) #Should be (200,), column vector length 200

(200, 1200)
(200,)


In [24]:
#Mean of X should be close to 0
np.mean(X[:,3])

0.007432079917791414

In [25]:
#Apply Lasso and check how many important  variables there are
model = Lasso(alpha=0.1, fit_intercept=False, max_iter=10000)
model.fit(X,y)
beta_hat = model.coef_

In [26]:
#Locations of non-zero variables in the predicted variable selection
print('Predicted variable selection found this many non-zero variables: ' +str(len(np.where(beta_hat != 0)[0])))
print('The data really has this many non-zero variables: ' +str(len(np.where(beta_star != 0)[0])))

Predicted variable selection found this many non-zero variables: 144
The data really has this many non-zero variables: 27


In [27]:
#This is the number of non-zero variables that were correctly found to be non-zero
len(np.intersect1d(np.where(beta_star != 0),np.where(beta_hat != 0)))

21

### April 1st

In [28]:
#for SKlearn compliance
from sklearn.base import BaseEstimator, RegressorMixin

In [29]:
class SQRTLasso(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=0.01):
        self.alpha = alpha

    def fit(self, x, y):
        alpha=self.alpha
        def f_obj(x,y,beta,alpha):
          n =len(x)
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = np.sqrt(1/n*np.sum((y-x.dot(beta))**2)) + alpha*np.sum(np.abs(beta))
          return output
        
        def f_grad(x,y,beta,alpha):
          n=x.shape[0]
          p=x.shape[1]
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = np.array((-1/np.sqrt(n))*np.transpose(x).dot(y-x.dot(beta))/np.sqrt(np.sum((y-x.dot(beta))**2))+alpha*np.sign(beta)).flatten()
          return output
        
        def objective(beta):
          return(f_obj(x,y,beta,alpha))
        
        def gradient(beta):
          return(f_grad(x,y,beta,alpha))
        
        beta0 = np.ones((x.shape[1],1))
        output = minimize(objective, beta0, method='L-BFGS-B', jac=gradient,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})
        beta = output.x
        self.coef_ = beta
        
    def predict(self, x):
        return x.dot(self.coef_)

In [30]:
n = 200
p = 1200

beta_star = np.concatenate(([1]*7,[0]*25,[0.25]*5,[0]*50,[0.7]*15,[0]*1098))

In [31]:
# we need toeplitz([1,0.8,0.8**2,0.8**3,0.8**4,...0.8**1199])
v = []
for i in range(p):
  v.append(0.8**i)

mu = [0]*p
sigma = 3.5
# Generate the random samples.
np.random.seed(123)
X = np.random.multivariate_normal(mu, toeplitz(v), size=n) # this where we generate some fake data
#y = X.dot(beta) + sigma*np.random.normal(0,1,[num_samples,1])
#y = np.matmul(x,beta_star).reshape(-1,1) + sigma*np.random.normal(0,1,size=(n,1))
y = X.dot(beta_star).reshape(-1,1) + sigma*np.random.normal(0,1,size=(n,1))

In [37]:
print(y)

[[-25.10551989]
 [  8.03463436]
 [ -7.96281493]
 [  5.21434154]
 [ -4.14046976]
 [ -4.79249863]
 [  8.88817664]
 [  1.57656237]
 [ -2.79026059]
 [  6.91599671]
 [-27.18721323]
 [ 23.66819477]
 [ -3.67123745]
 [  2.90373348]
 [  3.16303279]
 [  3.75313185]
 [ 14.50774285]
 [ -2.92310987]
 [ -3.05305551]
 [  1.70936055]
 [ -8.60997758]
 [ -1.95161352]
 [  3.26329159]
 [-15.4768899 ]
 [  5.26191791]
 [ 18.5829722 ]
 [-10.25654568]
 [  0.66591799]
 [ -2.06895739]
 [  9.35398165]
 [-13.46182646]
 [ -6.42117856]
 [ -4.22372637]
 [ -6.0364587 ]
 [ -2.5031809 ]
 [-25.73902325]
 [ -7.60374873]
 [ -5.98982352]
 [-13.66708144]
 [ -0.53020143]
 [ -7.87440299]
 [-23.27677577]
 [ 12.93668924]
 [ -8.66820021]
 [ -2.80100219]
 [-16.2908329 ]
 [ -3.90247275]
 [ -2.17318319]
 [  0.53738138]
 [ -4.60952518]
 [  2.29557674]
 [ -5.85836837]
 [ -1.68373837]
 [-11.83894279]
 [ 17.90857593]
 [  2.8944184 ]
 [  9.49527506]
 [ -8.98521124]
 [-13.10449558]
 [ -1.07272244]
 [  6.84258729]
 [ 17.56439614]
 [  4.99

In [33]:
model = SQRTLasso(alpha=0.1)

In [34]:
%%time
model.fit(X,y)
# This is split up to measure the fitting time

Wall time: 5.51 s


In [56]:
print(model.coef_)

[ 1.21828396e+00  1.49539244e+00  8.10656578e-01 ...  6.99434175e-07
 -1.59954043e-05 -2.62136574e-06]


In [69]:
#The L2 norm between the SQRTLasso coefficients and the ground truth
np.linalg.norm(model.coef_ - beta_star, ord=2)

1.209355633699586

In [58]:
model = Lasso(alpha=0.1)
model.fit(X,y)
beta_hat = model.coef_

In [70]:
# Being super slow, basically just optimizes hyperparameter values
grid = GridSearchCV(estimator=SQRTLasso(), cv=10, scoring='neg_mean_squared_error', param_grid={'alpha':np.linspace(0,1,50)})
grid.fit(X,y)

KeyboardInterrupt: 

In [62]:
#Want to design our own validation function and then apply a particular choice of grid search algorithms
def validate(model, X, y, nfolds, rs):
    from sklearn.model_selection import KFold, train_test_split as tts
    kf = KFold(n_splits=nfolds, shuffle=True, random_state=rs)
    PE = []
    for idxtrain, idxtest in kf.split(X):
        xtrain = X[idxtrain]
        ytrain = y[idxtrain]
        ytest = y[idxtest]
        xtest = X[idxtest]
        #xtrain = scale.fit_transform(xtrain)
        #xtest = scale.transform(xtest)
        model.fit(xtrain,ytrain)
        PE.append(mean_absolute_error(ytest,model.predict(xtest)))
    return np.mean(PE)


In [63]:
validate(model, X, y, 10, 123)

3.3325757456901606

In [64]:
model = SQRTLasso(alpha=0.2)
validate(model, X, y, 10, 123)

3.1803143534054126

In [68]:
model.fit(X,y)
np.linalg.norm(model.coef_ - beta_star, ord=2)

1.209355633699586

In [ ]:
model = SQRTLasso(alpha=0.1)
model.fit(X,y)
np.linalg.norm(model.coef_ - beta_star, ord=2)

In [71]:
# Genetic algorithm, also a hyperparamet tuner
!pip install geneticalgorithm

     ---------------------------------------- 44.3/44.3 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Created wheel for func-timeout: filename=func_timeout-4.3.5-py3-none-any.whl size=15096 sha256=7aaa3f8c9ab4a5e2949c10ed7efe064e73f1a00760d4d0b9e0c1bfd5e2554487
  Stored in directory: c:\users\mnelk\appdata\local\pip\cache\wheels\ef\91\3f\17cbea1e83d17cc01bd2d66a0d59e65301bcb2d59f2f6a62ab
Successfully built func-timeout


In [95]:
from geneticalgorithm import geneticalgorithm as ga

In [96]:
def objective(a):
    model=SQRTLasso(alpha=a)
    return validate(model,X,y,5,123)

In [97]:
varbound = np.array([[0,1]]*1)

opt = ga(function=objective, dimension = 1, variable_type='real', variable_boundaries=varbound)

opt.run()

given function is not applicable


AssertionError: After 10.0 seconds delay func_timeout: the given function does not provide any output